In [56]:
def kcal2kj (value):
    """
    convert unit from kcal/mol to kJ/mol 
    1 kcal = 4.184 kJ
    """

    return (float(value)*4.184)


def kj2kcal (value):
    """
    convert unit from kj/mol to kcal/mol 
    1 kcal = 4.184 kJ
    """
    
    return (float(value)/4.184)


def ev2kj (value):
    """
    convert unit from eV to kJ/mol 
    1 eV = 96 kJ/mol
    """
    
    return float(value)*96

In [12]:
with open("ffnonbonded.itp") as file:
    data = file.readlines()

atomtypes = dict()    
for line in data:
    line.strip(' ')
    if line[0] is not '[' and line[0] is not ';':
        word = line.split()
        atomtypes[word[0]] = word[1:8]

In [ ]:
with open("ffbonded.itp") as file:
    data = file.readlines()

i_bondtypes = data.index("[ bondtypes ]\n")
i_Atoms = data.index(" Atoms #full\n")
i_Bonds = data.index(" Bonds\n")
i_Angles = data.index(" Angles\n")
i_Dihedrals = data.index(" Dihedrals\n")

In [59]:
with open('ffbonded.itp') as file:
    ffbonded = file.readlines()
    
i_bondcoeff = ffbonded.index("[ bondtypes ]\n")
i_constraintypes = ffbonded.index("[ constrainttypes ]\n")
i_anglecoeff = ffbonded.index("[ angletypes ]\n")

bondcoeffdata = ffbonded[i_bondcoeff+1:i_constraintypes-1]
bondcoeff = dict()
for line in bondcoeffdata:
    if line[0] is not ';':
        word = line.split()
        bondcoeff[word[0] + " " + word[1]] = [float(word[3]),float(word[4])]


In [61]:
from collections import defaultdict

with open('topol.top') as file:
    data = file.readlines()
    
i_atoms = data.index("[ atoms ]\n")
i_bonds = data.index("[ bonds ]\n")
i_angles = data.index("[ angles ]\n")
i_dihedrals = data.index("[ dihedrals ]\n")
i_impropers = data.index("[ impropers ]\n")

atomdata = data[i_atoms+1:i_bonds-1]
bonddata = data[i_bonds+1:i_angles-1]
angleldata = data[i_angles+1:i_dihedrals-1]
dihedraldata = data[i_dihedrals+1:i_impropers-1]

atoms = dict()
for line in atomdata:
    word = line.split()
#     atoms[word[0]] = word[1]  # use atom name (opls_xxx)
    atoms[word[0]] = atomtypes[word[1]][0] # use bond_type name


bondtype = defaultdict(list)
c = 1 # c: type number
for line in bonddata:
    b = line.split()
    if (atoms[b[1]], atoms[b[0]]) not in bondtype.values() and (atoms[b[0]], atoms[b[1]]) not in bondtype.values():
        bondtype[c] = ((atoms[b[0]], atoms[b[1]]))
        c += 1
        
for i in sorted(bondtype.keys()):
    # look up bondcoeff
    try:
        [b0,kb] = bondcoeff[bondtype[i][0] + " " + bondtype[i][1]]
    except KeyError:
        [b0,kb] = bondcoeff[bondtype[i][1] + " " + bondtype[i][0]]
    print(f"{i} {bondtype[i]} {b0} {kb}")

1 ('HC', 'CT') 0.109 284512.0
2 ('CT', 'CT') 0.1529 224262.4
3 ('CT', 'C_2') 0.1522 265265.6
4 ('C_2', 'O_2') 0.1229 476976.0
5 ('C_2', 'OS') 0.1327 179075.2
6 ('OS', 'CT') 0.141 267776.0


In [46]:
bondtype.values()

dict_values([('HC', 'CT'), ('CT', 'CT'), ('CT', 'C_2'), ('C_2', 'O_2'), ('C_2', 'OS'), ('OS', 'CT')])